<a href="https://colab.research.google.com/drive/1vkEt3bYEi7pdqGTgyY1EVB1-zei1X4vA?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### Meta-Prompting: Task-Agnostic Scaffolding

In [2]:
!pip install -qU google-generativeai

In [3]:
import google.generativeai as genai
import getpass

Get free-tier Google's Gemini API Key here: https://aistudio.google.com/app/apikey

In [4]:
API_KEY = getpass.getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [6]:
genai.configure(api_key=API_KEY)

In [7]:
class MetaPromptingAgent:
    def __init__(self):
        self.model = genai.GenerativeModel("gemini-2.0-flash-exp")
        self.expert_library = self._default_experts()

    def _default_experts(self):
        """Library of virtual expert roles"""
        return {
            "Analyst": "Analyzes data, trends, and patterns objectively",
            "Engineer": "Provides technical and implementation perspectives",
            "Doctor": "Offers medical and health-related expertise",
            "Lawyer": "Understands legal, compliance, and regulatory issues",
            "Economist": "Analyzes financial, economic, and market aspects",
            "Designer": "Focuses on user experience, aesthetics, and usability",
            "Scientist": "Applies scientific methodology and research principles",
            "Psychologist": "Understands human behavior and mental processes",
            "Strategist": "Develops plans, strategies, and long-term thinking",
            "Ethicist": "Evaluates moral and ethical implications"
        }

    def decompose_task(self, query):
        """Break down complex problem into sub-tasks"""
        experts_list = "\n".join([f"- {name}: {desc}" for name, desc in self.expert_library.items()])

        prompt = f"""You are a Meta-Orchestrator. Break down this complex query into 2-4 sub-tasks.

        Query: {query}

        Available Experts:
        {experts_list}

        For each sub-task, specify:
        1. The sub-task description
        2. Which expert should handle it

        Format:
        Sub-task N: [description]
        Expert: [expert_name]

        Decomposition:"""

        response = self.model.generate_content(prompt).text
        return self._parse_decomposition(response)

    def _parse_decomposition(self, text):
        """Parse decomposition into structured format"""
        subtasks = []
        current_task = {}

        for line in text.split("\n"):
            line = line.strip()

            if line.startswith("Sub-task") or line.startswith("Subtask"):
                if current_task:
                    subtasks.append(current_task)
                # Extract description after colon
                if ":" in line:
                    current_task = {"description": line.split(":", 1)[1].strip()}
                else:
                    current_task = {"description": line}

            elif line.startswith("Expert:"):
                expert = line.split(":", 1)[1].strip()
                # Match expert name from library
                for expert_name in self.expert_library.keys():
                    if expert_name.lower() in expert.lower():
                        current_task["expert"] = expert_name
                        break

        if current_task:
            subtasks.append(current_task)

        return subtasks

    def simulate_expert(self, expert_name, subtask, original_query):
        """Simulate a virtual expert responding to sub-task"""
        expert_desc = self.expert_library.get(expert_name, "a general expert")

        prompt = f"""You are a virtual {expert_name}. {expert_desc}

        Original Query: {original_query}

        Your Specific Sub-task: {subtask}

        Provide your expert analysis and recommendations:"""

        response = self.model.generate_content(prompt).text
        return response.strip()

    def synthesize_responses(self, query, expert_responses):
        """Combine expert outputs into coherent answer"""
        responses_text = "\n\n".join([
            f"{expert_name} ({expert_desc}):\n{response}"
            for expert_name, expert_desc, response in expert_responses
        ])

        prompt = f"""Original Query: {query}

        Expert Responses:
        {responses_text}

        Synthesize these expert perspectives into a comprehensive, coherent answer:"""

        response = self.model.generate_content(prompt).text
        return response.strip()

    def solve(self, query):
        """Main meta-prompting pipeline"""
        print(f"\n{'='*60}")
        print(f"🎭 Meta-Prompting (Multi-Expert Simulation)")
        print(f"{'='*60}")
        print(f"Query: {query}\n")

        # Step 1: Decompose task
        print(f"{'─'*60}")
        print(f"STEP 1: Task Decomposition")
        print(f"{'─'*60}\n")

        subtasks = self.decompose_task(query)

        print(f"Identified {len(subtasks)} sub-tasks:\n")
        for i, task in enumerate(subtasks, 1):
            expert = task.get("expert", "Unknown")
            desc = task.get("description", "N/A")
            print(f"{i}. [{expert}] {desc}")
        print()

        # Step 2: Route to virtual experts
        print(f"{'─'*60}")
        print(f"STEP 2: Consulting Virtual Experts")
        print(f"{'─'*60}\n")

        expert_responses = []

        for i, task in enumerate(subtasks, 1):
            expert_name = task.get("expert")
            if not expert_name or expert_name not in self.expert_library:
                print(f"⚠️  Skipping task {i}: No valid expert assigned\n")
                continue

            subtask_desc = task.get("description", "")
            expert_desc = self.expert_library[expert_name]

            print(f"👤 Consulting {expert_name}...")
            print(f"   Task: {subtask_desc[:60]}...")

            response = self.simulate_expert(expert_name, subtask_desc, query)

            print(f"   Response: {response[:100]}...\n")

            expert_responses.append((expert_name, expert_desc, response))

        # Step 3: Synthesize
        print(f"{'─'*60}")
        print(f"STEP 3: Synthesizing Expert Responses")
        print(f"{'─'*60}\n")

        final_answer = self.synthesize_responses(query, expert_responses)

        print(f"{'='*60}")
        print(f"💡 FINAL ANSWER")
        print(f"{'='*60}")
        print(final_answer)
        print()

        return final_answer

In [ ]:
# Example 1: Multi-Domain Question
print("="*60)
print("EXAMPLE 1: Multi-Domain Question")
print("="*60)

agent1 = MetaPromptingAgent()
agent1.solve(
    "Should our company launch a new AI-powered health app? "
    "Consider technical feasibility, legal compliance, market opportunity, and ethical implications."
)


# Example 2: Complex Business Decision
print("\n" + "="*60)
print("EXAMPLE 2: Complex Business Decision")
print("="*60)

agent2 = MetaPromptingAgent()
agent2.solve(
    "We're considering a 4-day work week. Analyze the impact on productivity, "
    "employee wellbeing, financial costs, and legal requirements."
)


# Example 3: Product Development
print("\n" + "="*60)
print("EXAMPLE 3: Product Development Analysis")
print("="*60)

agent3 = MetaPromptingAgent()
agent3.solve(
    "Design a smart home security system. Consider engineering requirements, "
    "user experience design, privacy concerns, and market positioning."
)


# Example 4: Policy Evaluation
print("\n" + "="*60)
print("EXAMPLE 4: Policy Evaluation")
print("="*60)

agent4 = MetaPromptingAgent()
agent4.solve(
    "Evaluate a proposed carbon tax policy. Analyze economic impact, "
    "environmental benefits, legal framework, and public perception."
)


# Example 5: Healthcare Decision
print("\n" + "="*60)
print("EXAMPLE 5: Healthcare Strategy")
print("="*60)

agent5 = MetaPromptingAgent()
agent5.solve(
    "A hospital wants to implement AI diagnostics. Assess medical effectiveness, "
    "ethical considerations, legal liability, and implementation costs."
)


# Example 6: Creative Problem Solving
print("\n" + "="*60)
print("EXAMPLE 6: Creative Solution Development")
print("="*60)

agent6 = MetaPromptingAgent()
agent6.solve(
    "How can we reduce urban traffic congestion? "
    "Consider engineering solutions, economic incentives, policy changes, and behavioral psychology."
)


# Example 7: Technology Assessment
print("\n" + "="*60)
print("EXAMPLE 7: Technology Assessment")
print("="*60)

agent7 = MetaPromptingAgent()
agent7.solve(
    "Should we adopt blockchain for our supply chain? "
    "Evaluate technical capabilities, security implications, cost-benefit analysis, and strategic fit."
)


# Example 8: Custom Experts
print("\n" + "="*60)
print("EXAMPLE 8: Custom Expert Configuration")
print("="*60)

# Create agent with custom experts
custom_agent = MetaPromptingAgent()
custom_agent.expert_library = {
    "Marketing Expert": "Specializes in brand positioning and customer acquisition",
    "Data Scientist": "Analyzes data patterns and builds predictive models",
    "Operations Manager": "Optimizes processes and resource allocation",
    "Customer Success": "Understands user needs and satisfaction"
}

custom_agent.solve(
    "How can we improve customer retention for our SaaS product?"
)


print("✅ Meta-Prompting Complete!")

EXAMPLE 1: Multi-Domain Question

🎭 Meta-Prompting (Multi-Expert Simulation)
Query: Should our company launch a new AI-powered health app? Consider technical feasibility, legal compliance, market opportunity, and ethical implications.

────────────────────────────────────────────────────────────
STEP 1: Task Decomposition
────────────────────────────────────────────────────────────

Identified 4 sub-tasks:

1. [Economist] **Market Assessment and Financial Viability:** Analyze the market opportunity for an AI-powered health app, including market size, competitive landscape, potential revenue streams, and financial projections. Assess the economic viability of developing and launching the app.
2. [Engineer] **Technical Feasibility and Development Roadmap:** Evaluate the technical feasibility of building the proposed AI-powered health app. Define the necessary technology stack, infrastructure requirements, development timeline, and potential challenges. Consider data security, privacy, an